In [2]:
import urllib.parse
import urllib.request
import pandas as pd
import re
from tqdm.notebook import tqdm
import numpy as np
import pickle
import os


In [3]:
os.getcwd()
os.chdir("./data/")
os.getcwd()

'/home/hb/python/preprecessing3/data'

# 데이터 살펴보기
[1. Postmod](#Postmod)  
[2. Phospho.ELM](#PostPho.ELM)  
3. PSEA -> 접속불가  
[4. PhosphoNetworks](#PhosphoNetworks)    
5. HuPHO -> 접속불가  
[6. PhosphoPICK](#PhosphoPICK)  
7. RegPhos -> 접속불가    
8. PKIS -> 접속불가  
[9. EPSD](#EPSD)

dfPostMod = pd.read_csv('./PostMod.txt',encoding = 'ISO-8859-1', header = 0, delimiter = '\t')
display(dfPostMod.head()) #실험데이터 2816개
          #
          #PostMod by using Phospho.ELM database
dfPhosphoSitePlus = pd.read_csv('./PhosphoSitePlus.txt',encoding = 'ISO-8859-1', header = 0, delimiter = '\t')
display(dfPhosphoSitePlus.head()) #실험데이터

          
dfPhosphoELM = pd.read_csv('./Phospho.ELM.txt',encoding = 'ISO-8859-1', header = 0, delimiter = '\t')
display(dfPhosphoELM.head()) #실험데이터


dfPhosphoNetworks = pd.read_csv('./PhosphoNetworks.txt',encoding = 'ISO-8859-1', header = 0, delimiter = '\t')
display(dfPhosphoNetworks.head()) #실험데이터 3274개 UniProt와 Uniprot.1이 다르므로 kinase와 substrate로 구분가능할듯 
                  #datasets were generated from a collaborative project of 
                  #Drs. Heng Zhu, Jin Zhang, and Jiang Qian's labs 

dfDEPOD = pd.read_csv('./DEPOD.txt',encoding = 'ISO-8859-1', header = 0, delimiter = '\t')
display(dfDEPOD.head()) #실험데이터 데이터개수 265개
        
dfPhosphoPICK = pd.read_csv('./PhosphoPICK.txt',encoding = 'ISO-8859-1', header = 0, delimiter = '\t')
display(dfPhosphoPICK.head()) #예측 데이터 20209개 substrate에 대한 각각의 kinase별 p-value제공


dfNetworKIN = pd.read_csv('./NetworKIN.txt',encoding = 'ISO-8859-1', header = 0, delimiter = '\t')
display(dfNetworKIN.head()) #예측데이터 


dfGPS = pd.read_csv('./GPS.txt',encoding = 'ISO-8859-1', header = 0, delimiter = '\t')
display(dfGPS.head()) #예측데이터
dfEPSD = pd.read_csv('./EPSD.txt',encoding = 'ISO-8859-1', header = 0, delimiter = '\t')
display(dfEPSD.head())
dfGPSbench = pd.read_excel('./GPS5_benchmarkdata.xlsx',header = 3)
display(dfEPSD.head())        

## Postmod
- 추후 설명 추가하기

In [4]:
dfPostMod = pd.read_csv('./PostMod.txt',encoding = 'ISO-8859-1', header = 0, delimiter = '\t')
display(dfPostMod) #실험데이터 2816개
          #
          #PostMod by using Phospho.ELM database
        
dfPostMod.info()


,EPSD ID,UniProt,Position,Kinase,Source
0,EP0001281,O15162,Y69,Abl;Src,Experimental
1,EP0001281,O15162,Y74,Abl;Src,Experimental
2,EP0001340,O15350,Y99,Abl,Experimental
3,EP0006301,P00519,Y393,Abl;Src,Experimental
4,EP0006302,P00520,Y393,Abl,Experimental
...,...,...,...,...,...
2812,EP0009109,P12318,Y288,Fyn;Lyn;Syk;BLK,Experimental
2813,EP0009109,P12318,Y304,Fyn;Lyn;Syk;BLK,Experimental
2814,EP0016960,P51452,Y138,Syk;ZAP70,Experimental
2815,EP0012558,P31995,Y287,Syk,Experimental


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2817 entries, 0 to 2816
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   EPSD ID   2817 non-null   object
 1   UniProt   2817 non-null   object
 2   Position  2817 non-null   object
 3   Kinase    2817 non-null   object
 4   Source    2817 non-null   object
dtypes: object(5)
memory usage: 110.2+ KB


In [5]:
dfPostMod['Kinase'] = dfPostMod['Kinase'].apply(lambda x:x.split(';'))
dfPostMod = dfPostMod.explode('Kinase')
dfPostMod


,EPSD ID,UniProt,Position,Kinase,Source
0,EP0001281,O15162,Y69,Abl,Experimental
0,EP0001281,O15162,Y69,Src,Experimental
1,EP0001281,O15162,Y74,Abl,Experimental
1,EP0001281,O15162,Y74,Src,Experimental
2,EP0001340,O15350,Y99,Abl,Experimental
...,...,...,...,...,...
2813,EP0009109,P12318,Y304,BLK,Experimental
2814,EP0016960,P51452,Y138,Syk,Experimental
2814,EP0016960,P51452,Y138,ZAP70,Experimental
2815,EP0012558,P31995,Y287,Syk,Experimental


In [6]:
# EPSD ID = UniPort (substrate) 개수
print(f"substrate 개수", dfPostMod.iloc[:,0].value_counts(),sep='\n', end='\n\n') 

# dfPostMod의 'Kinase'이름의 열에서 고유값의 개수

# Kinase 개수
print(f"kinase 개수", dfPostMod.iloc[:,3].value_counts(),sep='\n', end='\n\n') 

# dfPostMod의 'Kinase'이름의 열에서 고유값의 개수)
print(f"예측값 or 실험값",dfPostMod.iloc[:,4].value_counts(),sep='\n', end='\n\n') 

# dfPostMod의 'Kinase'이름의 열에서 고유값의 개수)

substrate 개수
EP0010961    94
EP0009822    70
EP0025924    63
EP0008942    58
EP0009427    55
             ..
EP0021038     1
EP0009765     1
EP0012898     1
EP0015705     1
EP0018795     1
Name: EPSD ID, Length: 982, dtype: int64

kinase 개수
PKA_group           313
CK2_group           273
PKC_group           232
Src                 182
MAPK1               181
                   ... 
PKG1A/cGK-Ialpha      1
PAK3                  1
MARK_group            1
MAP3K_group           1
PYK2                  1
Name: Kinase, Length: 156, dtype: int64

예측값 or 실험값
Experimental    4458
Name: Source, dtype: int64



## Phospho.ELM 
http://phospho.elm.eu.org/dataset.html

Phospho.ELM version 9.0 contains 8,718 substrate proteins covering 3,370 tyrosine, 31,754 serine and 7,449 threonine instances (including data from high-throughput experiments).

The entries, manually annotated and based on scientific literature, provide information about the phosphorylated proteins and the exact position of known phosphorylated instances, the kinases responsible for the modification and links to bibliographic references. Additional information is also given about structure, interaction partners and sub-cellular compartment and tissue specificities when available.

The data set that we provide you with includes:

- UniPROT/Ensembl accession number
- sequence
- position
- phosphorylated residue
- PubMed Id
- the upstream kinase (when known)
- source (High-ThroughPut/Low-ThroughPut)
- species
- entry date

In [7]:
dfPhosphoELM = pd.read_csv('./Phospho.ELM.txt',encoding = 'ISO-8859-1', header = 0, delimiter = '\t')
display(dfPhosphoELM.head()) #실험데이터

        
dfPhosphoELM.info()

# EPSD ID = UniPort (substrate) 개수
print(f"substrate 개수", dfPhosphoELM.iloc[:,0].value_counts(),sep='\n', end='\n\n') 
# Kinase 개수
print(f"kinase 개수", dfPhosphoELM.iloc[:,3].value_counts(),sep='\n', end='\n\n') 
# Experimental or predicted?
print(f"예측값 or 실험값",dfPhosphoELM.iloc[:,4].value_counts(),sep='\n', end='\n\n') 


,EPSD ID,UniProt,Position,Kinase,Source
0,EP0000453,O08605,S39,PAK2,Experimental
1,EP0000453,O08605,T34,PAK2,Experimental
2,EP0001071,O14543,Y204,Lck,Experimental
3,EP0001071,O14543,Y221,Lck,Experimental
4,EP0001136,O14746,S824,PKB_group,Experimental


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4495 entries, 0 to 4494
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   EPSD ID   4495 non-null   object
 1   UniProt   4495 non-null   object
 2   Position  4495 non-null   object
 3   Kinase    4495 non-null   object
 4   Source    4495 non-null   object
dtypes: object(5)
memory usage: 175.7+ KB
substrate 개수
EP0009497    31
EP0023072    30
EP0006939    29
EP0013515    25
EP0025924    25
             ..
EP0030870     1
EP0006005     1
EP0007290     1
EP0089755     1
EP0015670     1
Name: EPSD ID, Length: 1251, dtype: int64

kinase 개수
PKA_group      297
PKC_group      213
CK2_group      206
PKC_alpha      163
CDK1           159
              ... 
NDR1             1
LRRK2            1
MST3             1
SIK              1
LIMK1_drome      1
Name: Kinase, Length: 284, dtype: int64

예측값 or 실험값
Experimental    4495
Name: Source, dtype: int64



## 3. PhosphoNetwork
To accomplish this task, we performed phosphorylation reactions with 289 human kinases on protein microarrays composed of 4,191 unique, full-length human proteins. We identified 3,656 kinase-substrate relationships that are likely to be physiologically relevant. We also determined phosphorylation motifs for 230 kinases and used these data to explore the complexity and diversity of motifs in the various kinase families.

- rawKSR : 24046 in-vitro KSR, identified by ptn microarray
- refKSR : 3656 refined, high-confidence relevant KSR
- comKSR : 744 literature-curated KSR + 3656 refKSR
- high-resolution map of KSR : we connected 230 kinases with 2591 phosphorylation sites on 652 substrate proteins.  (4839개)

In [36]:
HRN = pd.read_csv('./phosphonetworks/highResolutionNetwork.csv',encoding = 'ISO-8859-1', header = 0, delimiter = '\,')
display(HRN)



/home/hb/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,substrate,P-Site_Sequence,Human_P-Site,kinase,score
0,NaN,>ABCA1,NaN,NaN,NaN
1,NaN,SSYGISETTLEEIFL,T1242,CSNK2A1,1.0162
2,NaN,SYGISETTLEEIFLK,T1243,CSNK2A1,1.0455
3,NaN,IDPESRETDLLSGMD,T1305,CSNK2A1,1.0263
4,NaN,>ABI2,NaN,NaN,NaN
...,...,...,...,...,...
5064,NaN,QNQNQVRSPGAPGPL,S344,DMPK,1.2348
5065,NaN,QNQNQVRSPGAPGPL,S344,MAPK10,1.3287
5066,NaN,QNQNQVRSPGAPGPL,S344,RPS6KA2,2.7638
5067,NaN,PGAPGPLTLKEVEEL,T352,PRKAA2,2.3538


In [37]:
HRN['substrate'] = HRN['P-Site_Sequence'].apply(lambda x: x if ">" in x else np.nan)
HRN['substrate'] = HRN['substrate'].fillna(method="ffill")
HRN = HRN.dropna(axis=0)

,substrate,P-Site_Sequence,Human_P-Site,kinase,score
1,>ABCA1,SSYGISETTLEEIFL,T1242,CSNK2A1,1.0162
2,>ABCA1,SYGISETTLEEIFLK,T1243,CSNK2A1,1.0455
3,>ABCA1,IDPESRETDLLSGMD,T1305,CSNK2A1,1.0263
5,>ABI2,PPTQKPPSPPMSGKG,S177,CDK5,2.0987
6,>ABI2,PPTQKPPSPPMSGKG,S177,DYRK4,3.5732
...,...,...,...,...,...
5064,>ZYX,QNQNQVRSPGAPGPL,S344,DMPK,1.2348
5065,>ZYX,QNQNQVRSPGAPGPL,S344,MAPK10,1.3287
5066,>ZYX,QNQNQVRSPGAPGPL,S344,RPS6KA2,2.7638
5067,>ZYX,PGAPGPLTLKEVEEL,T352,PRKAA2,2.3538


In [88]:
display(HRN) #실험데이터
      
HRN.info()

# EPSD ID = UniPort (substrate) 개수
print(f"substrate 개수", HRN.iloc[:,0].value_counts(),sep='\n', end='\n\n') 
# Kinase 개수
print(f"kinase 개수", HRN.iloc[:,3].value_counts(),sep='\n', end='\n\n') 
# Experimental or predicted?
print(f"P-Site",HRN.iloc[:,2].value_counts(),sep='\n', end='\n\n') 


,substrate,P-Site_Sequence,Human_P-Site,kinase,score
1,>ABCA1,SSYGISETTLEEIFL,T1242,CSNK2A1,1.0162
2,>ABCA1,SYGISETTLEEIFLK,T1243,CSNK2A1,1.0455
3,>ABCA1,IDPESRETDLLSGMD,T1305,CSNK2A1,1.0263
5,>ABI2,PPTQKPPSPPMSGKG,S177,CDK5,2.0987
6,>ABI2,PPTQKPPSPPMSGKG,S177,DYRK4,3.5732
...,...,...,...,...,...
5064,>ZYX,QNQNQVRSPGAPGPL,S344,DMPK,1.2348
5065,>ZYX,QNQNQVRSPGAPGPL,S344,MAPK10,1.3287
5066,>ZYX,QNQNQVRSPGAPGPL,S344,RPS6KA2,2.7638
5067,>ZYX,PGAPGPLTLKEVEEL,T352,PRKAA2,2.3538


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4417 entries, 1 to 5068
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   substrate        4417 non-null   object 
 1   P-Site_Sequence  4417 non-null   object 
 2   Human_P-Site     4417 non-null   object 
 3   kinase           4417 non-null   object 
 4   score            4417 non-null   float64
dtypes: float64(1), object(4)
memory usage: 207.0+ KB
substrate 개수
>PPP1R13L    232
>ZYX         108
>NFATC4       98
>CTTN         88
>POLR2A       85
            ... 
>SERBP1        1
>SF1           1
>SF3B1         1
>SFMBT1        1
>RGS19         1
Name: substrate, Length: 652, dtype: int64

kinase 개수
MAPK1      234
CSNK2A1    230
MAPK3      196
CDK4       182
MAPK8      151
          ... 
ACVR1B       1
CDK16        1
EPHB3        1
NIM1         1
TRIB2        1
Name: kinase, Length: 230, dtype: int64

P-Site
S289     24
S302     21
S213     20
S113     19
S12

In [26]:
# i = 0 

# while i < len(HRN):
#     if ">" in HRN.iloc[i,1]:
#         HRN.iloc[i,0] == HRN.iloc[i,i]
#         while  ">" not in HRN.iloc[i+1,1]:
#             HRN.iloc[i+1,0] == HRN.iloc[i,0]
#             i = i+1
#             print(i)


KeyboardInterrupt: 

In [40]:
comKSI = pd.read_csv('./phosphonetworks/comKSI.csv',encoding = 'ISO-8859-1', header = 0, delimiter = '\t')
display(comKSI) #실험데이터
      
comKSI.info()

# EPSD ID = UniPort (substrate) 개수
print(f"substrate 개수", comKSI.iloc[:,1].value_counts(),sep='\n', end='\n\n') 
# Kinase 개수
print(f"kinase 개수", comKSI.iloc[:,0].value_counts(),sep='\n', end='\n\n') 


,Kinase,Substrate,Score
0,AAK1,SOX2,1.79
1,ABL2,CAT,100
2,ABL2,CBL,46.65
3,ABL2,GPX1,100
4,ABL2,HLCS,1.67
...,...,...,...
4370,ZAP70,PRKACA,15.18
4371,ZAP70,SH2B3,100
4372,ZAP70,SHC1,100
4373,ZAP70,TRAT1,6.03


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4375 entries, 0 to 4374
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Kinase     4375 non-null   object
 1   Substrate  4375 non-null   object
 2   Score      4375 non-null   object
dtypes: object(3)
memory usage: 102.7+ KB
substrate 개수
HLCS      75
MTA1      74
ZYX       59
CTTN      55
NFATC4    53
          ..
ZW10       1
ZNF655     1
NCOA1      1
PAK1       1
EPHB3      1
Name: Substrate, Length: 1140, dtype: int64

kinase 개수
STK17A     110
PRKCA      109
AKT1       100
CSNK2A1     95
CDK4        92
          ... 
PKN3         1
SGK2         1
SGK3         1
MST4         1
AAK1         1
Name: Kinase, Length: 255, dtype: int64



## PosphoPICK
http://bioinf.scmb.uq.edu.au/phosphopick/download

In [41]:
dfPhosphoPICK = pd.read_csv('./PhosphoPICK.txt',encoding = 'ISO-8859-1', header = 0, delimiter = '\t')
display(dfPhosphoPICK) #예측 데이터 20209개 substrate에 대한 각각의 kinase별 p-value제공

dfPhosphoPICK.info()

# EPSD ID = UniPort (substrate) 개수
print(f"substrate 개수", dfPhosphoPICK.iloc[:,0].value_counts(),sep='\n', end='\n\n') 

# dfPostMod의 'Kinase'이름의 열에서 고유값의 개수

# Kinase 개수
print(f"kinase 개수", dfPhosphoPICK.iloc[:,2].value_counts(),sep='\n', end='\n\n') 

print(f"예측값 or 실험값",dfPhosphoPICK.iloc[:,4].value_counts(),sep='\n', end='\n\n') 

,EPSD ID,UniProt,Kinase,p-value,Source
0,EP0012552,P31946,JNK1,0.010188,Predicted
1,EP0012552,P31946,CDK7,0.018099,Predicted
2,EP0012552,P31946,GSK3A,0.044990,Predicted
3,EP0012552,P31946,DYRK1A,0.021773,Predicted
4,EP0012552,P31946,CDK9,0.032500,Predicted
...,...,...,...,...,...
244954,EP0025550,Q12436,IPL1,0.009484,Predicted
244955,EP0013117,P34240,TPK1,0.028453,Predicted
244956,EP0013117,P34240,TPK3,0.018820,Predicted
244957,EP0013117,P34240,IPL1,0.009484,Predicted


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244959 entries, 0 to 244958
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   EPSD ID  244959 non-null  object 
 1   UniProt  244959 non-null  object 
 2   Kinase   244959 non-null  object 
 3   p-value  244959 non-null  float64
 4   Source   244959 non-null  object 
dtypes: float64(1), object(4)
memory usage: 9.3+ MB
substrate 개수
EP0006301    87
EP0006717    86
EP0006306    85
EP0007533    84
EP0012534    84
             ..
EP0073999     1
EP0073393     1
EP0031130     1
EP0030911     1
EP0012735     1
Name: EPSD ID, Length: 38428, dtype: int64

kinase 개수
NEK2    24047
Hck     24047
IPL1     4967
ERK2     2715
Src      2708
        ...  
PKH2      298
HOG1      298
FUS3      295
SLT2      294
FRK1      291
Name: Kinase, Length: 133, dtype: int64

예측값 or 실험값
Predicted    244959
Name: Source, dtype: int64



## PhosphoSitePlus

In [43]:

dfPhosphoSitePlus = pd.read_csv('./PhosphoSitePlus.txt',encoding = 'ISO-8859-1', header = 0, delimiter = '\t')
display(dfPhosphoSitePlus) #실험데이터
dfPhosphoSitePlus.info()


,EPSD ID,UniProt,Position,Kinase,Source
0,EP0021246,Q01986,S298,PAK2,Experimental
1,EP0087333,Q9WTQ1,S203,PAK2,Experimental
2,EP0006440,P01237,S206,PAK2,Experimental
3,EP0007082,P05198,S52,HRI,Experimental
4,EP0007082,P05198,S49,HRI,Experimental
...,...,...,...,...,...
12281,EP0001211,O14974,S910,NuaK1,Experimental
12282,EP0006005,O95835,S464,NuaK1,Experimental
12283,EP0001230,O15027,S668,ULK2,Experimental
12284,EP0134296,A2RUS2,S472,ULK2,Experimental


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12286 entries, 0 to 12285
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   EPSD ID   12286 non-null  object
 1   UniProt   12286 non-null  object
 2   Position  12286 non-null  object
 3   Kinase    12286 non-null  object
 4   Source    12286 non-null  object
dtypes: object(5)
memory usage: 480.0+ KB


In [45]:


# EPSD ID = UniPort (substrate) 개수
print(f"substrate 개수", dfPhosphoSitePlus.iloc[:,0].value_counts(),sep='\n', end='\n\n') 

# dfPostMod의 'Kinase'이름의 열에서 고유값의 개수

# Kinase 개수
print(f"kinase 개수", dfPhosphoSitePlus.iloc[:,3].value_counts(),sep='\n', end='\n\n') 

print(f"예측값 or 실험값",dfPhosphoSitePlus.iloc[:,4].value_counts(),sep='\n', end='\n\n') 

substrate 개수
EP0006939    98
EP0013382    56
EP0041750    51
EP0007248    51
EP0022179    50
             ..
EP0043576     1
EP0045185     1
EP0001101     1
EP0002514     1
EP0018087     1
Name: EPSD ID, Length: 3239, dtype: int64

kinase 개수
PKACA         649
CK2A1         549
ERK2          517
CDK1          507
PKCA          498
             ... 
BRD4            1
TIE1            1
WEE1B           1
PKACA iso2      1
GRK7            1
Name: Kinase, Length: 386, dtype: int64

예측값 or 실험값
Experimental    12286
Name: Source, dtype: int64



In [48]:
# dfDEPOD = pd.read_csv('./DEPOD.txt',encoding = 'ISO-8859-1', header = 0, delimiter = '\t')
# display(dfDEPOD.head()) #실험데이터 데이터개수 265개
# # kinase에 상반되는 개념인 Phophatase 정보         


## NetworKIN 
https://networkin.info/

In [49]:
dfNetworKIN = pd.read_csv('./NetworKIN.txt',encoding = 'ISO-8859-1', header = 0, delimiter = '\t')
display(dfNetworKIN) #예측데이터 

,EPSD ID,UniProt,Position,Kinase,NetworKIN score,Tree,Group,NetPhorest score,STRING identifier,STRING score,Substrate name,Sequence,STRING path,Source
0,EP0080070,Q9NQ94,Y401,Abl2,0.9937,SH2,Abl_group,0.0713,ENSP00000282641,0.0930,A1CF,LGGRGyLAYTG,"ENSP00000356595, 0.192 ENSP00000316333, 0.64 E...",Predicted
1,EP0080070,Q9NQ94,Y401,BLK,1.1800,SH2,Src_group,0.1525,ENSP00000282641,0.1342,A1CF,LGGRGyLAYTG,"ENSP00000259089, 0.7248 ENSP00000363115, 0.627...",Predicted
2,EP0080070,Q9NQ94,Y401,BMX,1.0033,SH2,Tec_group,0.0754,ENSP00000282641,0.1076,A1CF,LGGRGyLAYTG,"ENSP00000308774, 0.7792 ENSP00000300086, 0.401...",Predicted
3,EP0080070,Q9NQ94,Y401,BTK,1.0033,SH2,Tec_group,0.0754,ENSP00000282641,0.1346,A1CF,LGGRGyLAYTG,"ENSP00000308176, 0.7248 ENSP00000363115, 0.627...",Predicted
4,EP0080070,Q9NQ94,Y401,Brk,0.9864,SH2,Brk,0.0680,ENSP00000282641,0.1041,A1CF,LGGRGyLAYTG,"ENSP00000217185, 0.4264 ENSP00000313829, 0.688...",Predicted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357828,EP0002359,O43149,T1521,CDK3,0.5244,KIN,CDK2_CDK3_CDK1_CDK5_group,0.3553,ENSP00000371051,0.1145,ZZEF1,PLSPStPTRRP,"ENSP00000400088, 0.7936 ENSP00000345571, 0.538...",Predicted
357829,EP0002359,O43149,T1521,DAPK3,1.0000,KIN,DAPK_group,0.1121,ENSP00000371051,0.1226,ZZEF1,PLSPStPTRRP,"ENSP00000301264, 0.6736 ENSP00000261207, 0.538...",Predicted
357830,EP0002359,O43149,T1521,MAPK1,1.8959,KIN,MAPK3_MAPK1_MAPK7_NLK_group,0.2835,ENSP00000371051,0.1527,ZZEF1,PLSPStPTRRP,"ENSP00000215832, 0.7192 ENSP00000268763, 0.568...",Predicted
357831,EP0002359,O43149,T1521,MAPK3,1.7579,KIN,MAPK3_MAPK1_MAPK7_NLK_group,0.2835,ENSP00000371051,0.1464,ZZEF1,PLSPStPTRRP,"ENSP00000263025, 0.7192 ENSP00000268763, 0.568...",Predicted


In [53]:
dfNetworKIN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357833 entries, 0 to 357832
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   EPSD ID            357833 non-null  object 
 1   UniProt            357833 non-null  object 
 2   Position           357833 non-null  object 
 3   Kinase             357833 non-null  object 
 4   NetworKIN score    357833 non-null  float64
 5   Tree               357833 non-null  object 
 6   Group              357833 non-null  object 
 7   NetPhorest score   357833 non-null  float64
 8   STRING identifier  357833 non-null  object 
 9   STRING score       357833 non-null  float64
 10  Substrate name     357833 non-null  object 
 11  Sequence           357833 non-null  object 
 12  STRING path        357833 non-null  object 
 13  Source             357833 non-null  object 
dtypes: float64(3), object(11)
memory usage: 38.2+ MB


In [56]:


# EPSD ID = UniPort (substrate) 개수
print(f"substrate 개수", dfNetworKIN.iloc[:,0].value_counts(),sep='\n', end='\n\n') 

# dfPostMod의 'Kinase'이름의 열에서 고유값의 개수

# Kinase 개수
print(f"kinase 개수", dfNetworKIN.iloc[:,3].value_counts(),sep='\n', end='\n\n') 

print(f"예측값 or 실험값",dfNetworKIN.iloc[:,13].value_counts(),sep='\n', end='\n\n') 

substrate 개수
EP0084333    2812
EP0015841    1781
EP0013513    1727
EP0088335    1627
EP0008770    1506
             ... 
EP0063959       1
EP0065562       1
EP0064501       1
EP0018360       1
EP0088329       1
Name: EPSD ID, Length: 6488, dtype: int64

kinase 개수
PAK1          11925
CK1alpha       9330
PKCbeta        9274
Yes            8472
PKCalpha       8431
              ...  
DAPK2            24
CaMKIgamma       15
MRCKb            14
LOK               9
DMPK2             5
Name: Kinase, Length: 207, dtype: int64

예측값 or 실험값
Predicted    357833
Name: Source, dtype: int64



## GPS
http://gps.biocuckoo.cn/  


In [86]:
dfGPS_non = pd.read_csv('./GPS.txt',encoding = 'ISO-8859-1', header = 0, delimiter = '\t')
display(dfGPS_non) #예측데이터

,EPSD ID,UniProt,Position,Kinase,Peptide,Score,Cutoff,Source
0,EP0168130,F4HRS2,S2059,CK1,EPDLPGRSSIQDDES,8.194,5.872,Predicted
1,EP0168130,F4HRS2,S2060,TKL,PDLPGRSSIQDDESN,4.606,4.354,Predicted
2,EP0168130,F4HRS2,S2060,TKL,PDLPGRSSIQDDESN,4.606,4.354,Predicted
3,EP0168130,F4HRS2,S2060,AGC/PKA,PDLPGRSSIQDDESN,4.866,4.418,Predicted
4,EP0168130,F4HRS2,S2047,Atypical/TAF1,GSESKTESVAEEEPD,4.750,4.681,Predicted
...,...,...,...,...,...,...,...,...
3818875,EP0050570,Q6Q595,S136,AGC,VLPKRKPSGNHDAES,1.933,1.600,Predict
3818876,EP0050570,Q6Q595,S136,STE,VLPKRKPSGNHDAES,2.654,1.285,Predict
3818877,EP0050570,Q6Q595,S136,AGC,VLPKRKPSGNHDAES,1.933,1.600,Predict
3818878,EP0050570,Q6Q595,S136,STE,VLPKRKPSGNHDAES,2.654,1.285,Predict


In [80]:
dfGPS = pd.read_csv('./GPS.txt',encoding = 'ISO-8859-1', header = 0, delimiter = '\t')
display(dfGPS) #예측데이터

,EPSD ID,UniProt,Position,Kinase,Peptide,Score,Cutoff,Source
0,EP0168130,F4HRS2,S2059,CK1,EPDLPGRSSIQDDES,8.194,5.872,Predicted
1,EP0168130,F4HRS2,S2060,TKL,PDLPGRSSIQDDESN,4.606,4.354,Predicted
2,EP0168130,F4HRS2,S2060,TKL,PDLPGRSSIQDDESN,4.606,4.354,Predicted
3,EP0168130,F4HRS2,S2060,AGC/PKA,PDLPGRSSIQDDESN,4.866,4.418,Predicted
4,EP0168130,F4HRS2,S2047,Atypical/TAF1,GSESKTESVAEEEPD,4.750,4.681,Predicted
...,...,...,...,...,...,...,...,...
3818875,EP0050570,Q6Q595,S136,AGC,VLPKRKPSGNHDAES,1.933,1.600,Predict
3818876,EP0050570,Q6Q595,S136,STE,VLPKRKPSGNHDAES,2.654,1.285,Predict
3818877,EP0050570,Q6Q595,S136,AGC,VLPKRKPSGNHDAES,1.933,1.600,Predict
3818878,EP0050570,Q6Q595,S136,STE,VLPKRKPSGNHDAES,2.654,1.285,Predict


In [81]:
dfGPS['Kinase'] = dfGPS['Kinase'].apply(lambda x: x.split("/"))
dfGPS = dfGPS.explode('Kinase')

In [84]:
dfGPS

,EPSD ID,UniProt,Position,Kinase,Peptide,Score,Cutoff,Source
0,EP0168130,F4HRS2,S2059,CK1,EPDLPGRSSIQDDES,8.194,5.872,Predicted
1,EP0168130,F4HRS2,S2060,TKL,PDLPGRSSIQDDESN,4.606,4.354,Predicted
2,EP0168130,F4HRS2,S2060,TKL,PDLPGRSSIQDDESN,4.606,4.354,Predicted
3,EP0168130,F4HRS2,S2060,AGC,PDLPGRSSIQDDESN,4.866,4.418,Predicted
3,EP0168130,F4HRS2,S2060,PKA,PDLPGRSSIQDDESN,4.866,4.418,Predicted
...,...,...,...,...,...,...,...,...
3818876,EP0050570,Q6Q595,S136,STE,VLPKRKPSGNHDAES,2.654,1.285,Predict
3818877,EP0050570,Q6Q595,S136,AGC,VLPKRKPSGNHDAES,1.933,1.600,Predict
3818878,EP0050570,Q6Q595,S136,STE,VLPKRKPSGNHDAES,2.654,1.285,Predict
3818879,EP0050570,Q6Q595,S136,AGC,VLPKRKPSGNHDAES,5.313,4.418,Predict


In [85]:
# EPSD ID = UniPort (substrate) 개수
print(f"substrate 개수", dfGPS.iloc[:,0].value_counts(),sep='\n', end='\n\n') 

# dfPostMod의 'Kinase'이름의 열에서 고유값의 개수

# Kinase 개수
print(f"kinase 개수", dfGPS.iloc[:,3].value_counts(),sep='\n', end='\n\n') 

print(f"예측값 or 실험값",dfGPS.iloc[:,7].value_counts(),sep='\n', end='\n\n') 

substrate 개수
EP0084333    13772
EP0058329    12073
EP0104505     9190
EP0067164     8168
EP0133994     7265
             ...  
EP0075167        1
EP0018300        1
EP0183898        1
EP0004220        1
EP0052784        1
Name: EPSD ID, Length: 92320, dtype: int64

kinase 개수
CMGC     736006
AGC      712420
CAMK     659088
Other    511285
TKL      371243
          ...  
FGFR       4640
Ret        3830
Axl        3702
Tie        3689
Ack        1348
Name: Kinase, Length: 90, dtype: int64

예측값 or 실험값
Predicted    6796789
Predict        42044
Name: Source, dtype: int64



In [87]:

# EPSD ID = UniPort (substrate) 개수
print(f"substrate 개수", dfGPS_non.iloc[:,0].value_counts(),sep='\n', end='\n\n') 

# dfPostMod의 'Kinase'이름의 열에서 고유값의 개수

# Kinase 개수
print(f"kinase 개수", dfGPS_non.iloc[:,3].value_counts(),sep='\n', end='\n\n') 

print(f"예측값 or 실험값",dfGPS_non.iloc[:,7].value_counts(),sep='\n', end='\n\n') 

substrate 개수
EP0084333    7570
EP0058329    6635
EP0104505    5024
EP0067164    4643
EP0133994    4088
             ... 
EP0011470       1
EP0103732       1
EP0078562       1
EP0078561       1
EP0001429       1
Name: EPSD ID, Length: 92320, dtype: int64

kinase 개수
CMGC         201200
CMGC/DYRK    153590
CMGC/GSK     135274
CAMK         126802
STE          118452
              ...  
TK/FGFR        4640
TK/Ret         3830
TK/Axl         3702
TK/Tie         3689
TK/Ack         1348
Name: Kinase, Length: 89, dtype: int64

예측값 or 실험값
Predicted    3795481
Predict        23399
Name: Source, dtype: int64



## DEPOD data 살펴보기 (0323 교수님과 의논한 부분)

In [6]:
import pandas as pd
import os

In [7]:
os.getcwd()

'/home/hb/python/preprecessing3'

In [9]:

dfDEPOD = pd.read_csv('./data/DEPOD.txt',encoding = 'ISO-8859-1', header = 0, delimiter = '\t')
display(dfDEPOD) #실험데이터 데이터개수 265개
dfDEPOD.info()

,EPSD ID,UniProt,Position,Phosphatase,Source
0,EP0006932,P04626,NaN,ACPP,Experimental
1,EP0017167,P52333,NaN,CD45,Experimental
2,EP0012155,P29597,1054,CD45,Experimental
3,EP0012155,P29597,1055,CD45,Experimental
4,EP0057392,Q86WV1,232,CD45,Experimental
...,...,...,...,...,...
260,EP0006932,P04626,877,VHR,Experimental
261,EP0006932,P04626,1221,VHR,Experimental
262,EP0018670,P60484,380,PTEN,Experimental
263,EP0018670,P60484,382,PTEN,Experimental


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   EPSD ID      265 non-null    object
 1   UniProt      265 non-null    object
 2   Position     194 non-null    object
 3   Phosphatase  265 non-null    object
 4   Source       265 non-null    object
dtypes: object(5)
memory usage: 10.5+ KB


In [ ]:
df1 = pd.read_excel('DEPOD.xls', header = 0)
df1